In [1]:
# Python scripts with the implementation of AIMA of the classes
# "Problem" and "Node" and the search algorithms.
from search_mod import *

## Definition of the problem

We extend the class `Problem` implemented by AIMA to define the problem for this task, called: `MinningProblem`. To achieve this, we define the following methods:

- `__init__(self, initial, goal)`: This method initializes the problem with the initial state and the goal state.
- `is_valid_state(self, state)`: This method checks if a given state is valid, taking into account the number of rows and columns of the problem.
- `actions(self, state)`: This method returns the possible actions that can be taken from a given state. The actions will consists on the next states that can be reached from the current state.
- `result(self, state, action)`: This method returns the state that results from taking an action from a given state (in this case, the action is the next state).
- `h(self, node)`: This method returns the heuristic value of a given node.


In [4]:
class MinningProblem(Problem):
    """"""

    def __init__(self, rows, columns, initial, goal):
        self.rows, self.columns = rows, columns
        self.initial, self.goal = initial, goal

    def is_valid_state(self, state):
        """
        Check that the given state is a possible
        state given a board with rows and columns
        """
        return all(x > 0 for x in state) and state[0] < self.rows and state[1] < self.columns and state[2] < 8

    def actions(self, state):
        """Return the possible movements of the robot given a state"""
        # Get the orientation of the robot
        orientation = state[2]
        # We always can rotate the robot 45 degrees to the left and to the right
        movements = [(state[0], state[1], (orientation + 1) % 8), (state[0], state[1], (orientation - 1) % 8)]

        # The startegy to move forward the robot is to move
        # it always and then check if new_state is valid
        # Calculate the new position on axis y
        new_state = list(state)
        if orientation >= 7 or orientation <= 1:
            new_state[0] = state[0] - 1
        elif orientation >= 3 and orientation <= 5:
            new_state[0] = state[0] + 1
        # Calculate the new position on axis x
        if orientation >= 1 and orientation <= 3:
            new_state[1] = state[1] + 1
        elif orientation >= 5 and orientation <= 7:
            new_state[1] = state[1] - 1

        if self.is_valid_state(new_state):
            movements.append(tuple(new_state))

        return movements

    def result(self, state, action):
        """Move the robot to the next state. The action consists in the next state"""
        return action

    def h1(self, node):
        """Count only the positions between the actual state and the goal state (Manhattan heuristic)"""
        return abs(self.goal[0] - node.state[0]) + abs(self.goal[1] - node.state[1])

    def h2(self, node):
        """Manhattan heuristic taking into account the actual orientation of the robot"""
        orientation = node.state[2]
        # Calculate the number of movements on axis y that the robot has to
        # go up or down. If the value is positive, the robot has to go down
        # and if it is negative, it has to go up
        movements_y = self.goal[0] - node.state[0]
        orientation_y = 4 if movements_y > 0 else 0 if movements_y < 0 else orientation

        # Calculate the number of movements on axis y that the robot has to
        # go right or left
        movements_x = self.goal[1] - node.state[1]
        orientation_x = 2 if movements_x > 0 else 6 if movements_x < 0 else orientation

        # Calculate the number of rotations that are needed
        # to move forward to the goal state in both axis
        number_rotations_y = (orientation - orientation_y) % 8
        number_rotations_x = (orientation - orientation_x) % 8

        print(
            f"Orientation: {orientation}.",
            f"Goal: {self.goal}.",
            f"Orientation y: {orientation_y}.",
            f"Orientation x: {orientation_x}.",
        )
        print(f"Rotations y: {number_rotations_y}.")
        print(f"Rotations x: {number_rotations_x}.")

        # step 1: rotate the robot to orientate it in the direction of the nearest axis
        number_rotations_step_1 = min(number_rotations_y, number_rotations_x)
        # step 2: rotate the robot to orientate it in the other axis
        number_rotations_step_2 = (number_rotations_y - number_rotations_x) % 8
        number_rotations = number_rotations_step_1 + number_rotations_step_2

        total_cost = abs(movements_x) + abs(movements_y) + number_rotations

        if number_rotations_y < number_rotations_x:
            actions = (
                orientation,
                orientation_y,
                number_rotations_y,
                abs(movements_y),
                "y",
                orientation_x,
                number_rotations_step_2,
                abs(movements_x),
                "x",
            )
        else:
            actions = (
                orientation,
                orientation_x,
                number_rotations_x,
                abs(movements_x),
                "x",
                orientation_y,
                number_rotations_step_2,
                abs(movements_y),
                "y",
            )

        print(
            f"Actions:\n- First, rotate robot from orientation {actions[0]} to {actions[1]} (cost: {actions[2]}).\n"
            + f"- Then move {actions[3]} steps on axis {actions[4]} (cost: {actions[3]}).\n"
            + f"- Rotate robot from orientation {actions[1]} to {actions[5]} (cost: {actions[6]}).\n"
            + f"- Move {actions[7]} steps on axis {actions[8]} (cost: {actions[7]}).\n"
            + f"- Total cost: {total_cost}."
        )

        return total_cost

    def h(self, node):
        return self.h2(node)

In [3]:
mp = MinningProblem(3, 4, (0, 3, 0), (2, 0, 8))

mp.actions((0, 3, 7))

[(0, 3, 0), (0, 3, 6)]

In [62]:
print(mp.h1(Node((0, 3, 0))))
print(mp.h2(Node((0, 3, 0))))

print(mp.h1(Node((0, 3, 2))))
print(mp.h2(Node((0, 3, 2))))

5
Actions:
- First, rotate robot from orientation 0 to 6 (cost: 2).
- Then move 3 steps on axis x (cost: 3).
- Rotate robot from orientation 6 to 4 (cost: 2).
- Move 2 steps on axis y (cost: 2).
- Total cost: 9.
9
5
Actions:
- First, rotate robot from orientation 2 to 6 (cost: 4).
- Then move 3 steps on axis x (cost: 3).
- Rotate robot from orientation 6 to 4 (cost: 2).
- Move 2 steps on axis y (cost: 2).
- Total cost: 11.
11
